In [44]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec

In [45]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'c64e61f377b247abb392344165e7b192', 'Date': 'Tue, 27 Aug 2024 23:49:11 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Thompson',
  'subject': 'Calculus I',
  'stars': 5,
  'review': 'Dr. Thompson explains concepts very clearly and is always willing to help during office hours. Highly recommend!'},
 {'professor': 'Prof. John Rodriguez',
  'subject': 'Introduction to Psychology',
  'stars': 4,
  'review': 'Interesting lectures and engaging material, but grading can be a bit tough.'},
 {'professor': 'Dr. Emily Carter',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': 'Challenging course with a lot of material. Dr. Carter is knowledgeable, but the exams are very difficult.'},
 {'professor': 'Prof. Mark Davidson',
  'subject': 'World History',
  'stars': 4,
  'review': 'Great storyteller! Makes history come alive, though there’s a lot of reading required.'},
 {'professor': 'Dr. Linda Nguyen',
  'subject': 'Microeconomics',
  'stars': 2,
  'review': 'The class material is interesting, but Dr. Nguyen’s lectures can be monotonous and hard to follow.'},
 {'professor': 'Prof

In [ ]:
path=os.getenv('REVIEWS_PATH') or 'reviews.json'

In [46]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [39]:
# Initialize the model

processed_data = []
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

for review in data['reviews']:
    # Generate content using the correct method
    response = genai.embed_content(
        model='models/text-embedding-004',
        content=review['review'])  # Pass the review text directly
    
    # Assuming the response contains generated content in 'text'
    embedding= response['embedding']
    # Append the processed data
    processed_data.append({
        "values": embedding,  # Store generated content instead of embedding
        "id": review["professor"],
        "metadata": {
            "review": review['review'],
            "subject": review['subject'],
            "stars": review['stars']
        }
    })

In [40]:
processed_data[0]

{'values': [0.04446709,
  0.020971274,
  -0.029832818,
  -0.024944497,
  0.03304297,
  0.015123308,
  0.020934967,
  0.025812574,
  -0.005447346,
  0.0007523953,
  0.05411577,
  0.03885251,
  0.008305826,
  0.02971597,
  -0.025267093,
  -0.03101154,
  -0.04874166,
  0.05469439,
  -0.106189385,
  0.021231443,
  0.008984524,
  -0.036787767,
  -0.020348165,
  -0.0063105095,
  0.012377521,
  -0.017286515,
  0.016126981,
  -0.068515815,
  0.04060888,
  -0.0129129505,
  0.022993026,
  0.002164014,
  -0.0389044,
  -0.04536143,
  -0.03247828,
  -0.00816243,
  0.026159829,
  -0.018447824,
  0.016669268,
  -0.020743601,
  0.01282545,
  -0.04337507,
  -0.05741982,
  -0.030325038,
  -0.027826918,
  -0.023606306,
  -0.03865116,
  0.09380388,
  0.020340566,
  0.060534097,
  0.05059371,
  0.01166907,
  -0.06055669,
  0.041022222,
  -0.0322865,
  -0.027510626,
  -0.06140357,
  -0.058656696,
  0.08382946,
  -9.162779e-05,
  -0.05576658,
  -0.018783314,
  -0.02001912,
  -0.08377653,
  0.054471575,
  -0.

In [41]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 20}

In [42]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}